In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import layers, Model, optimizers, models

In [ ]:
data_path = '/path/to/the/features/folder/'

In [ ]:
x_train = np.load(os.path.join(data_path, 'features_univ_clinic_train.npy'))
x_val = np.load(os.path.join(data_path, 'features_univ_clinic_val.npy'))
x_test = np.load(os.path.join(data_path, 'features_univ_clinic_test.npy'))

y_train = np.load(os.path.join(data_path, 'features_univ_derm_train.npy'))
y_val = np.load(os.path.join(data_path, 'features_univ_derm_val.npy'))
y_test = np.load(os.path.join(data_path, 'features_univ_derm_test.npy'))

In [ ]:
def plot_loss(m, title):
    plt.plot(m.history['loss'])
    plt.plot(m.history['val_loss'])
    plt.title(title)
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper right')
    plt.grid()
    plt.show()

In [ ]:
def original_mmse(clinical, derm):
    error = clinical-derm
    sq_error = np.power(error,2)
    mse = sq_error.mean(axis=1)
    mmse = mse.mean()
    return mmse

In [ ]:
tf.config.list_physical_devices('GPU')

In [ ]:
input_layer = layers.Input(shape=(2048,), name='clinic_input')
fcn1 = layers.Dense(1024, activation='relu', name='fcn1')(input_layer)
d1 = layers.Dropout(0.25)(fcn1)
fcn2 = layers.Dense(512, activation='relu', name='fcn2')(d1)
d2 = layers.Dropout(0.25)(fcn2)
fcn3 = layers.Dense(512, activation='relu', name='fcn3')(d2)
d3 = layers.Dropout(0.25)(fcn3)
fcn4 = layers.Dense(1024, activation='relu', name='fcn4')(d3)
d4 = layers.Dropout(0.25)(fcn4)
output_layer = layers.Dense(2048, activation='relu', name='output')(d4)

modelx = Model(inputs=input_layer, outputs=output_layer, name='guidance')

sgd = optimizers.SGD(learning_rate=0.5, decay=1e-6, momentum=0.9, nesterov=True)
modelx.compile(loss='mse', optimizer=sgd)
print(modelx.summary())

In [ ]:
m = modelx.fit(x_train, y_train, batch_size=100, epochs=500, validation_data=(x_val, y_val), verbose=0)

In [ ]:
plot_loss(m, 'model')

In [ ]:
test_loss = modelx.evaluate(x_test, y_test, batch_size=50, verbose=0)

In [ ]:
save_path = '/path/to/guidance/model'
modelx.save(save_path)